### lxml Implementation

In [19]:
from pathlib import Path
from lxml import etree as et

file_read = Path.cwd() / "files" / "processes" / "input" / "AccountProcess.process"

with open(file_read, "r+") as f:
    tree = et.parse(f)

root = tree.getroot()
root

<Element {http://xmlns.tibco.com/bw/process/2003}ProcessDefinition at 0x105eab580>

### Remove Namespaces using XSLT Transformation

In [20]:
xslt_doc = Path.cwd() / "files" / "common" / "xslt_doc.xml"
with open(xslt_doc, "r+") as f:
    xslt = et.parse(f)

transform = et.XSLT(xslt)
result = transform(root)

transformed_doc = Path.cwd() / "files" / "processes" / "generated" / "transformed_doc.xml"
result.write(transformed_doc)

### Method to convert Tibco Mappings into Python Dictionary

In [21]:
def get_mapping(mapper) -> dict:
    mappings = mapper.find("./inputBindings/root/")
    my_dict = {}

    slicer = len(mappings.xpath("ancestor::*"))

    for var in mappings.xpath(".//value-of"):
        # Build the full_path from inputBindings with slicing logic
        ancestors_list = var.xpath("ancestor::*")  # excludes self
        full_path = "/".join(
            ancestor.tag for ancestor in ancestors_list[slicer:] if ancestor.tag not in ("if"))

        my_dict[full_path] = var.attrib["select"].strip('"')

    return my_dict

### Method to dump JSON contents into a file

In [22]:
import json

def write_json(content, path):
    with open(path, "w", encoding="utf-8") as json_file:
        json.dump(content, json_file, ensure_ascii=False, indent=4)

### Spot the Tibco-Mapper activity to retrieve the Tibco Mapping Logic

In [23]:
activities = result.findall("./activity")

mapper_list = []
for activity in activities:
    if activity.findtext("resourceType") == "ae.activities.MapperActivity":
        mapper_list.append(activity)

dict_path = Path.cwd() / "files" / "processes" / "generated" / "mapping_dictionary.json"
mapper_dict = {}
if len(mapper_list) == 1:
    mapper_dict = get_mapping(mapper_list[0])
write_json(mapper_dict, dict_path)